# Groundwater Flow example

This example is taken from Beskos et al (2016) 'Geometric MCMC for infinite-dimensional inverse problems', and the notation largely follows their section 4, and we commented where we deviated from it.

In [ ]:
import torch 
import time
import numpy as np
import random
import sys
from scipy.stats import norm
from scipy.stats import geom
from torch import nn
from copy import deepcopy as dc
import numba

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['text.usetex'] = True
matplotlib.use('Agg')

torch.autograd.set_detect_anomaly(False) #makes code very slow but one can find errors in backward()

sigma = 0.01                        # noise, called sigma_y in the cited paper
dim = 2                             # dimensionality of the space (v: R^d --> R)
obs_min = np.array([0,0])
obs_max = np.array([1,1])

In [ ]:
"""

initialise NN

"""

# Hyperparameters for our network
input_size = 2*dim
output_size = 1
n_layer = 1

In [ ]:
"""

FUNCTIONS - PRIOR

""" 


''' Set sigma_w and sigma_b uniformly '''    
alpha = 1.001  
sigma_w_sq = np.ones(n_layer+1)*100
sigma_b_sq = np.ones(n_layer+1)*100
sigma_w_sq[-1] = 1/30
sigma_b_sq[-1] = 1/10


''' prior samples are represented by array of coefficients '''
def sample_prior(c=1):
    w = []
    b = []
    
    ''' weights '''
    w_layer = np.random.randn(model[0].weight.shape[0],model[0].weight.shape[1])*C_root[0][0]
    w.append(c*w_layer)
    
    for l in range(1,n_layer):
        w_layer = np.random.randn(model[2*l].weight.shape[0],model[2*l].weight.shape[1])*C_root[0][l]
        w.append(c*w_layer)
     
    w_layer = np.random.randn(model[2*n_layer].weight.shape[0],model[2*n_layer].weight.shape[1])*C_root[0][-1]
    w.append(c*w_layer)
                
    ''' biases '''
    b_layer = np.random.randn(model[0].bias.shape[0])*C_root[1][0]
    b.append(c*b_layer)
    
    for l in range(1,n_layer):
        b_layer = np.random.randn(model[2*l].bias.shape[0])*C_root[1][l]
        b.append(c*b_layer)
     
    b_layer = np.random.randn(model[2*n_layer].bias.shape[0])*C_root[1][-1]
    b.append(c*b_layer)
    
    return [w,b]


''' evaluate the log_prior up to a constant '''
def logprior(xi):
    log_prior = 0
    w = xi[0]
    b = xi[1]
    
    ''' weights '''
    for l in range(n_layer+1):
        log_prior += -0.5*np.sum(w[l]**2/C[0][l])
        
    ''' biases '''
    for l in range(n_layer+1):
        log_prior += -0.5*np.sum(b[l]**2/C[1][l])
        
    return log_prior


def prior_proposal(xi):
    log_prior_xi = logprior(xi)
    
    w = xi[0]
    b = xi[1]
    
    ''' Firstly make a cpoy of the current state '''
    w_proposal = []
    b_proposal = []
    for l in range(n_layer+1):
        w_proposal.append(dc(w[l]))
        b_proposal.append(dc(b[l]))
    
    random_l = np.random.randint(n_layer)
    
    ''' Sample index of function to be swapped '''
    random_i = geom.rvs(1/alpha)
    while random_i>=model[2*random_l].weight.shape[0]-1:  # -1???
        random_i = geom.rvs(1/alpha)
        
    ''' swap f^l_i with f^l_{i+1}'''
    w_proposal[random_l][random_i,:] = w[random_l][random_i+1,:]
    w_proposal[random_l][random_i+1,:] = w[random_l][random_i,:]
    w_proposal[random_l+1][:,random_i] = w[random_l+1][:,random_i+1]
    w_proposal[random_l+1][:,random_i+1] = w[random_l+1][:,random_i]
    b_proposal[random_l][random_i] = b[random_l][random_i+1]
    b_proposal[random_l][random_i+1] = b[random_l][random_i]
    
    xi_proposal = [w_proposal,b_proposal]
    log_prior_proposal = logprior(xi_proposal)
    
    a = np.exp(log_prior_proposal-log_prior_xi)
    if np.random.uniform() < a:
        counts[0] += 1
        return xi_proposal
    else:
        counts[1] += 1
        return xi
    
counts = np.zeros(2)

In [ ]:
"""

FUNCTIONS - LIKELIHOOD 

"""

m = 21


def loglikelihood(y):
    k = np.zeros(m**2)
    for j in range(m):
        for i in range(m):
            k[i+m*j] = u((i/(m-1),j/(m-1))).detach().numpy()[0]
    LL = -1/sigma**2*np.dot(k-y,k-y)/2
    return LL

In [ ]:
"""

FUNCTIONS - VALUE FUNCTION

"""

''' u(x), which evaluates the function u at x '''
def u(x):
    x_transformed = np.zeros(dim)
    for i in range(dim):
        x_transformed[i] = ((x[i]-obs_min[i])/(obs_max[i]-obs_min[i])-1/2)*2
        
    x_transformed_again = np.zeros(2*dim)
    for i in range(dim):
        x_transformed_again[i] = x_transformed[i]
        x_transformed_again[dim+i] = np.sin(x_transformed[i]*2*np.pi)
        
    value = model(torch.from_numpy(x_transformed_again).float())
    return value

In [ ]:
"""

FUNCTIONS - ANALYTICS

"""
    
''' Progress bar to know how much longer one has to wait '''
def progressBar(t,value, t_max, acceptances, bar_length=40):
    percent = float(t) / t_max
    arrow = '-' * int(round(percent * bar_length)-1) + '>'
    spaces = ' ' * (bar_length - len(arrow))
    sys.stdout.write("\rIteration: {0}    Acceptance ratio: {1}    Percent: [{2}] {3}%  ".format(value,round(acceptances/value,3),arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush()    
        
''' Plotting a permeability '''    
def func_plot(xi,name):
    x = np.arange(0,1,0.005)
    y = np.arange(0,1,0.005)
    X,Y = np.meshgrid(x,y)
    Z = np.zeros(X.shape)
    
    for l in range(n_layer+1):
        model[2*l].weight = torch.nn.Parameter(torch.from_numpy(xi[0][l]).float(), requires_grad=False)
        model[2*l].bias = torch.nn.Parameter(torch.from_numpy(xi[1][l]).float(), requires_grad=False)
        
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i,j] = np.exp(u((X[i,j],Y[i,j]))[0])

    fig, ax = plt.subplots()
    c = ax.pcolormesh(X, Y, Z, cmap='cool', vmin=Z.min(), vmax=Z.max())
    
    # set the limits of the plot to the limits of the data
    ax.axis([X.min(), X.max(), Y.min(), Y.max()])
    fig.colorbar(c, ax=ax)
    fig.savefig(name + '.pdf', dpi=300, bbox_inches='tight')
    plt.close(fig)
    
''' Plotting a log permeability '''    
def func_plot_log(xi,name):
    x = np.arange(0,1,0.005)
    y = np.arange(0,1,0.005)
    X,Y = np.meshgrid(x,y)
    Z = np.zeros(X.shape)
    
    for l in range(n_layer+1):
        model[2*l].weight = torch.nn.Parameter(torch.from_numpy(xi[0][l]).float(), requires_grad=False)
        model[2*l].bias = torch.nn.Parameter(torch.from_numpy(xi[1][l]).float(), requires_grad=False)
        
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i,j] = u((X[i,j],Y[i,j]))[0]

    fig, ax = plt.subplots()
    c = ax.pcolormesh(X, Y, Z, cmap='cool', vmin=-0.7, vmax=0.6)
    
    # set the limits of the plot to the limits of the data
    ax.axis([X.min(), X.max(), Y.min(), Y.max()])
    fig.colorbar(c, ax=ax)
    fig.savefig(name + '.pdf', dpi=300, bbox_inches='tight')
    plt.close(fig)
    
''' Plot a trajectory '''  
def trajectory_plot(xi,name):
    x = np.arange(len(xi))
    fig = plt.figure()
    plt.plot(x,xi)
    fig.savefig(name + '.png', bbox_inches='tight')
    plt.close(fig)
    
''' Compute the autocorrelations '''    
def autocorr(x,lags):
    mean=np.mean(x)
    var=np.var(x)
    xp=x-mean
    corr=[1. if l==0 else np.sum(xp[l:]*xp[:-l])/len(x)/var for l in lags]
    return np.array(corr)

''' Calculate the Effective Sample Size, assumes algorithm already burned in '''
def ESS(logposterior,name):
    fig, ax = plt.subplots()
    N = len(logposterior)
    ax.stem(autocorr(logposterior, range(int(N*0.1))),use_line_collection=True) 
    ESS = N/(1+2*sum(autocorr(logposterior, range(int(N*0.1)))))
    print('Effective Sample Size:', round(ESS))
    print('Samples required to generate 1 independent sample:', round(N/ESS,2))
    fig.savefig(name + '.png', bbox_inches='tight')
    plt.close(fig)    

In [ ]:
"""

FUNCTIONS - MCMC (pCN/pCNL)

"""

def acceptance_prop(xi_u, xi_v,data,ll_u,diff_u=False):
    accept_prop = -ll_u
    
    for l in range(n_layer+1):
        model[2*l].weight = torch.nn.Parameter(torch.from_numpy(xi_v[0][l]).float(), requires_grad=True)
        model[2*l].bias = torch.nn.Parameter(torch.from_numpy(xi_v[1][l]).float(), requires_grad=True)
        
    ll_v = loglikelihood(data)
    accept_prop += ll_v
    
    return min(1, np.exp(accept_prop)),ll_v
    
def propose(xi,diff=False):
    w = xi[0]
    b = xi[1]
    
    noise = sample_prior()
    w_noise = noise[0]
    b_noise = noise[1]
    
    w_proposal = []
    b_proposal = []
    for l in range(n_layer+1):
        w_proposal.append(np.zeros(model[2*l].weight.shape))
        b_proposal.append(np.zeros(model[2*l].bias.shape))
    
    for l in range(n_layer+1):
        w_proposal[l] = np.sqrt(1-beta*beta)*w[l]+beta*w_noise[l]
        b_proposal[l] = np.sqrt(1-beta*beta)*b[l]+beta*b_noise[l]
    return [w_proposal,b_proposal]

def MCMC(xi,N_data,data,max_time):   
    print('\nMCMC algorithm ('+method + ', N_data=' + str(N_data) + ', ' + str(max_time) + ' seconds) was started: ' + str(time.ctime()))
      
    acc_ratio = 0
    logposterior = []
    logp = []
    logl = []
    
    ''' Set model weights and biases to current iterate '''
    for l in range(n_layer+1):
        model[2*l].weight = torch.nn.Parameter(torch.from_numpy(xi[0][l]).float(), requires_grad=True)
        model[2*l].bias = torch.nn.Parameter(torch.from_numpy(xi[1][l]).float(), requires_grad=True)
        
    ''' Initialise likelihood and gradient '''    
    ll = loglikelihood(data)
    print('Initial loglikelihood: ',ll)
        
    ''' Run MCMC '''
    start = time.time() 
    j = 0
    it = 0
    while(time.time()-start<max_time):
        
        ''' Swap functions around to get mode switching '''
        for _ in range(hidden_sizes[0]):
            xi = prior_proposal(xi)
        
        ''' Propose and calculate acceptance probability '''
        xi_proposal = propose(xi)  
        a,ll_proposal = acceptance_prop(xi,xi_proposal,data,ll)
        
        ''' Accept or reject proposal '''
        uni = np.random.uniform()
        if uni < a:
            xi = xi_proposal    
            ll = ll_proposal
            acc_ratio = acc_ratio + 1

        ''' prior, likelihood, and posterior traceplots are appended '''
        lp = logprior(xi)
        logposterior.append(lp+ll)
        logp.append(lp)
        logl.append(ll)
        
        if (j+1)%100==0:
            progressBar(time.time()-start,j+1,max_time,acc_ratio)
        j+=1
        
    progressBar(max_time,j,max_time,acc_ratio)
    
    acc_ratio = acc_ratio/(j)
    print('\nMCMC algorithm terminated: ' + str(time.ctime()) + '. \nRuntime = ' + str(time.time()-start) + '\nSteps: ' + str(j))
    print('Final loglikelihood: ',ll)
    print('Acceptance ratio is ',acc_ratio)
    
    trajectory_plot(logposterior[1:],'figs/GWF_no_forward_model/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_logposterior')
    trajectory_plot(logp[1:],'figs/GWF_no_forward_model/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_logprior')
    trajectory_plot(logl[1:],'figs/GWF_no_forward_model/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_loglikelihood')
    
    for l in range(n_layer+1):
        np.save('np_saved/GWF_no_forward_model/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample_w'+str(l)+'.npy',xi[0][l])
        np.save('np_saved/GWF_no_forward_model/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample_b'+str(l)+'.npy',xi[1][l])
    func_plot_log(xi,'figs/GWF_no_forward_model/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample')
    
    ESS(logposterior,'figs/GWF_no_forward_model/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_autocorr')

# MAIN PROGRAMMES

In [ ]:
"""

MAIN PROGRAMME 1

"""    

# set maximal runtime
t_max = 4*24*3600

''' Initialise network network, see second block for detailed comments '''
input_size = 2*dim
output_size = 1
n_layer = 1
hidden_sizes = [100]
hyps = str(hidden_sizes[0])
for i in range(1,n_layer):
    hyps = hyps+'_'+str(hidden_sizes[i])
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.Tanh(),
                      nn.Linear(hidden_sizes[-1], output_size))
weights = []
biases = []
for l in range(n_layer+1):
    weights.append(np.zeros(model[2*l].weight.shape))
    biases.append(np.zeros(model[2*l].bias.shape))
for l in range(n_layer+1):
    model[2*l].weight = torch.nn.Parameter(torch.from_numpy(0*weights[l]).float(), requires_grad=False)
    model[2*l].bias = torch.nn.Parameter(torch.from_numpy(0*biases[l]).float(), requires_grad=False)
''' Initialise covariance operator, see prior block for detailed comments '''
C = [[],[]]
C_root = [[],[]]
C_arr = np.ones(model[0].weight.shape)
for t in range(hidden_sizes[0]):
    for s in range(input_size):
        C_arr[t][s] = sigma_w_sq[0]/np.power(t+1,alpha)
C[0].append(C_arr)
C_root[0].append(C_arr**(1/2))
for l in range(1,n_layer):
    C_arr = np.ones(model[2*l].weight.shape)
    for t in range(hidden_sizes[l]):
        for s in range(hidden_sizes[l-1]):
            C_arr[t][s] = sigma_w_sq[l]/np.power(s+1,alpha)/np.power(t+1,alpha)
    C[0].append(C_arr)
    C_root[0].append(C_arr**(1/2))
C_arr = np.ones(model[2*n_layer].weight.shape)   
for t in range(output_size):
    for s in range(hidden_sizes[n_layer-1]):
        C_arr[t][s] = sigma_w_sq[n_layer]/np.power(s+1,alpha)
C[0].append(C_arr)   
C_root[0].append(C_arr**(1/2))
C_arr = np.ones(model[0].bias.shape)
for t in range(hidden_sizes[0]):
    C_arr[t] = sigma_b_sq[0]/np.power(t+1,alpha)
C[1].append(C_arr)
C_root[1].append(C_arr**(1/2))
for l in range(1,n_layer):
    C_arr = np.ones(model[2*l].bias.shape)
    for t in range(hidden_sizes[l]):
        C_arr[t] = sigma_b_sq[l]/np.power(t+1,alpha)
    C[1].append(C_arr)
    C_root[1].append(C_arr**(1/2))
C_arr = np.ones(model[2*n_layer].bias.shape)
for t in range(output_size):
    C_arr[t] = sigma_b_sq[n_layer]/np.power(t+1,alpha)
C[1].append(C_arr)
C_root[1].append(C_arr**(1/2))

# Sample from the prior to see what a sample looks like
xi = sample_prior()
func_plot(xi,'figs/GWF_no_forward_model/NN_'+hyps+'_a_prior_sample')

N_data = m**2
data = np.load('GWF_data_all_u.npy')

''' run pCN '''
# np.random.seed(42)
method = 'pCN'
beta =  1/2500
counts = np.zeros(2)
try:
    xi = [[],[]]
    for l in range(n_layer+1):
        xi[0].append(np.load('np_saved/GWF_no_forward_model/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample_w'+str(l)+'.npy'))
        xi[1].append(np.load('np_saved/GWF_no_forward_model/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample_b'+str(l)+'.npy'))
except FileNotFoundError:
    print('Starting from close to 0')
    xi = sample_prior(0.01)

MCMC(xi,N_data,data,t_max) 
print('Counts: '+str(int(counts[0]))+' accepted prior moves, '+str(int(counts[1]))+' rejected ones.')  

print('\n\n\nSo far this code ran 25 full days.')